In [5]:
import os
import math
import pickle
from datetime import datetime
import time as t
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve
from matplotlib.ticker import FormatStrFormatter

In [6]:
# for simulation
def coef_openloop(rl, eps, lam, tl, nn):
    N = len(rl)
    hr = rl[1]-rl[0]

    # 定义常数
    c1 = eps * tl / (2 * hr**2)
    c2 = eps * tl / (4 * hr)
    c3 = tl * lam / 2
    c4 = eps * tl * nn**2 / 2
    # 第一行系数 在起始点处（即i=0情况下，i-1项系数刚好为0）
    a1 = 1 + 2 * c1 - c3[0] + 4 * c4 / (hr**2)
    a2 = - c1 - 2 * c2 / hr
    a3 = 1 - 2 * c1 + c3[0] - 4 * c4 / (hr**2)
    a4 = c1 + 2 * c2 / hr

    # 初始化系数矩阵的稀疏结构
    data_at = [a1,a2]
    column_at = [0,1]
    offset_at = [0]
    data_a = [a3,a4]
    column_a = [0,1]
    offset_a = [0]

    rrl = rl[1:-1] #去掉边界点 因为第一行比较特殊（进行了单独计算避免了维度设置），最后一行施加控制律。

    #构建At矩阵系数
    At_1 = -(c1 * np.ones(N-2)) + c2 / rrl   # for i-1
    At_2 = (1 + 2 * c1) * np.ones(N-2) - c3[1:-1] + c4 / (rrl**2)  # for i
    At_3 = -(c1 * np.ones(N-2)) - c2 / rrl 

    #对于每一行，记录非零元素的列索引和对应值
    for i in range(1,N-1):
        # 记录行偏移位置
        offset_at.append(len(data_at))
        # 第一部分下对角线
        data_at.append(At_1[i-1])
        column_at.append(i-1)

        # 第二部分对角线
        data_at.append(At_2[i-1])
        column_at.append(i)

        # 第三部分上对角线
        data_at.append(At_3[i-1])
        column_at.append(i+1)
    
    # 定义边界条件
    offset_at.append(len(data_at)) # 最后一行的第一个元素是即将添加的第一个元素，该元素在data中的索引为data的长度。data长度为n则下一个元素的索引为n(实际为第n+1的元素)
    data_at.append(1)
    column_at.append(N-1)
    offset_at.append(len(data_at)) # 行偏移向量中的最后一个元素是非零元素的个数

    # 构建At矩阵
    At = csr_matrix((data_at, column_at, offset_at), shape=(N, N))

    # 构建A矩阵系数
    A_1 = c1 * np.ones(N-2) - c2 / rrl
    # A_2 = (1 - 2 * c1) * np.ones(N-2) + c2 / rrl + c3[1:-1] - c4 / (rrl**2)
    A_2 = (1 - 2 * c1) * np.ones(N-2) + c3[1:-1] - c4 / (rrl**2)
    A_3 = c1 * np.ones(N-2) + c2 / rrl

    #对于每一行，记录非零元素的列索引和对应值
    for i in range(1,N-1):
        # 记录行偏移位置
        offset_a.append(len(data_a))
        # 第一部分下对角线
        data_a.append(A_1[i-1])
        column_a.append(i-1)

        # 第二部分对角线
        data_a.append(A_2[i-1])
        column_a.append(i)

        # 第三部分上对角线
        data_a.append(A_3[i-1])
        column_a.append(i+1)
    
    # 定义边界条件
    offset_a.append(len(data_a)) # 最后一行的第一个元素是即将添加的第一个元素，该元素在data中的索引为data的长度。data长度为n则下一个元素的索引为n(实际为第n+1的元素)
    data_a.append(1)
    column_a.append(N-1)
    offset_a.append(len(data_a)) # 行偏移向量中的最后一个元素是非零元素的个数（即data的长度），该向量的元素个数为行数+1.

    A = csr_matrix((data_a, column_a, offset_a), shape=(N, N))

    return(At, A)

def coef_closed_loop(rl, eps, lam, tl, nn, al, kl):
    N = len(rl)
    hr = rl[1]-rl[0]

    # 定义常数
    c1 = eps * tl / (2 * hr**2)
    c2 = eps * tl / (4 * hr)
    c3 = tl * lam / 2
    c4 = eps * tl * nn**2 / 2
    # 第一行系数
    a1 = 1 + 2 * c1 - c3[0] + 4 * c4 / (hr**2)
    a2 = -c1 - 2 * c2 / hr
    a3 = 1 - 2 * c1 + c3[0] - 4 * c4 / (hr**2)
    a4 = c1 + 2 * c2 / hr

    # 计算控制系数
    Atc_n = -1/2 * al *kl
    Atc_n[0][-1] = -Atc_n[0][-1]/2 +1
    Ac_n = 1/2 * al *kl
    

    # 初始化系数矩阵的稀疏结构
    data_at = [a1,a2]
    column_at = [0,1]
    offset_at = [0] # 存储的每一个数字表示对应行的第一个非零元素在data中的索引，第一个数字表示第一行，第二个数字表示第二行数据从date中的第几个元素索引。
    data_a = [a3,a4]
    column_a = [0,1]
    offset_a = [0]
    # 去掉边界点,便于计算可以循环构造矩阵部分的系数
    rrl = rl[1:-1]
    #构建At矩阵系数
    At_1 = -(c1 * np.ones(N-2)) + c2 / rrl   # for i-1
    At_2 = (1 + 2 * c1) * np.ones(N-2) - c3[1:-1] + c4 / (rrl**2)  # for i
    At_3 = -c1 * np.ones(N-2) - c2 / rrl 

    #对于每一行赋值
    for i in range(1,N-1):
        # 记录行偏移位置 即这一行第一个数据在data中的索引号
        offset_at.append(len(data_at))
        # 第一部分下对角线
        data_at.append(At_1[i-1]) # 在data中添加数据 取数组中第i-1个元素是因为循环是从1开始的（因为第一行是手动设置），而数组中的索引是从0开始的（只包含可以循环添加的部分）。
        column_at.append(i-1) # 记录列向量位置 i-1是因为是下对角线，i为行数，i-1为列数

        # 第二部分对角线
        data_at.append(At_2[i-1])
        column_at.append(i)

        # 第三部分上对角线
        data_at.append(At_3[i-1])
        column_at.append(i+1)
    
    # 定义边界条件控制律
    # 在python中索引是从0开始，则对于即将被添加的数据索引为“添加前数组的长度”，长度为n，索引为n，但实际为第n+1个元素（刚好为新增）。
    offset_at.append(len(data_at)) 
    
    for i in range(0,N):
        data_at.append(Atc_n[0][i])
        column_at.append(i)
    
    offset_at.append(len(data_at)) # 行偏移向量中的最后一个元素是非零元素的个数

    # 构建At矩阵
    At = csr_matrix((data_at, column_at, offset_at), shape=(N, N))

    # 构建A矩阵系数
    A_1 = c1 * np.ones(N-2) - c2 / rrl
    A_2 = (1 - 2 * c1) * np.ones(N-2) + c3[1:-1] - c4 / (rrl**2)
    A_3 = c1 * np.ones(N-2) + c2 / rrl

    #对于每一行，记录非零元素的列索引和对应值
    for i in range(1,N-1):
        # 记录行偏移位置
        offset_a.append(len(data_a))
        # 第一部分下对角线
        data_a.append(A_1[i-1])
        column_a.append(i-1)

        # 第二部分对角线
        data_a.append(A_2[i-1])
        column_a.append(i)

        # 第三部分上对角线
        data_a.append(A_3[i-1])
        column_a.append(i+1)
    
    # 定义边界条件
    offset_a.append(len(data_a)) # 最后一行的第一个元素是即将添加的第一个元素，该元素在data中的索引为data的长度。data长度为n则下一个元素的索引为n(实际为第n+1的元素)
    
    for i in range(0,N):
        data_a.append(Ac_n[0][i])
        column_a.append(i)

    offset_a.append(len(data_a)) # 行偏移向量中的最后一个元素是非零元素的个数

    A = csr_matrix((data_a, column_a, offset_a), shape=(N, N))

    return(At, A)

def Simpson_constant(N, hr):
    # hr = delta_r
    # 根据simpson离散规则，必须离散为偶数个区间，则离散点数量必须为奇数（N）
    # 根据索引的奇偶性，对数组进行赋值
    # start : stop : step 从start开始，到不包括stop的位置，步长为step
    constant = np.ones((1,N))*2 # 生成一个二维数组
    constant[0,1:N-1:2] = 4 
    # 对边界点进行赋值
    constant[0,0] = 1
    constant[0,N-1] = 1
    # 
    val = hr/3*constant
    return val


In [7]:
# define the ordinary parameters
c =1 # Stability Index
R_max =1 # spatial distance
eps =1 # diffusion coefficient
final_time = 7 # final time
t_step = 0.0005
Max_step = int(final_time/t_step) # calculate the number of time steps

# optional parameters
N =91 # number of grid points
D = 1 # convergence coefficient

# discretize the spatial and time domain
delta_r = (2*R_max)/(2*N-1)
r = np.arange(1, N+1, dtype=np.float64) # np.arange(start, stop, step, dtype=...)
r =(r-1/2)*delta_r
time = np.arange(0, final_time+t_step, t_step)
time = time[1:] # remove the t=0 initial condition that don't need to simulate

# define reaction coefficient
def reaction_coef_normal(r, a):
    # only choose one of the following
    # lam = 40 - 20*np.cos(np.pi*r)
    constant =a
    lam = constant*r**2
    # print(f"reaction coefficient is: {constant}*r^2")
    return lam

def reaction_coef_cos(r, a=40, b=20):
    lam = a - b*np.cos(np.pi*r)
    cl = [-b, b, -b, b, -b,b]
    a0z = a
    # print(f"reaction coefficient is: {a}-{b}*cos(pi*r)")
    return lam, cl, a0z

# 计算需要控制的幂级数范围
def cal_power_range(lamda_max, eps, D, R_max):
    # calculate the power range
    n_max = ((R_max**2*(D + 2*lamda_max))/(2*eps)-1/4)**0.5
    n_max = math.floor(n_max)  # 向上取整
    power_range = np.arange(0, n_max+1)
    return power_range

# # check
# print("unit spatial step is:", delta_r, "\ntotal spatial steps:", len(r), "\nunit time step is:", time[1]-time[0], "\ntotal time steps:", len(time))
# print("max reaction coefficient is:", lamda_max)
# print("power range is:", n)

In [8]:
import solve_kernel as sk
num_lamda = 10000
dataset =[]
counter = 0
start_time = t.time()

for j in range(num_lamda):
    reaction_constant =np.random.uniform(45, 65)
    lamda = reaction_coef_normal(r, reaction_constant)
    # [lamda, cl, a0z] = reaction_coef_cos(r)
    lamda_max = np.max(lamda)

    n = cal_power_range(lamda_max, eps, D, R_max)
    nl = len(n)

# range(nl)生成从0到nl-1的整数序列
    for i in range(nl):
        # print(f"power range is: {n[i]}")

        # initialize the simulation condition
        # 生成一个多维数组，用于存储状态
        u = np.zeros((N, Max_step))
        a = Simpson_constant(N, delta_r)

        # 计算kernel (only choose one)
        # [k, kernel] = sk.getK_cos(r, n[i], cl, a0z, c)
        [K, kernel] = sk.getK(r, n[i])

        sample = {
            "lamda": lamda.astype(np.float32),
            "n": int(n[i]),
            "Kernel": kernel.real.astype(np.float32),
        }
        dataset.append(sample)

        counter += 1
    if j % 1000 == 0:
        elapsed_time = t.time() - start_time
        print(f"⏱️ {counter} samples have been generated (reaction coefficient is: {reaction_constant}*r^2). Time: {elapsed_time:.2f} s")
        # 计算At和A
        # [At, A] = coef_closed_loop(r, eps, lamda, t_step, n[i], a, K)

        # # initial condition
        # u_initial = 10 * np.ones(N)
        # u_initial[-1] = 0
        # u[:,0] = u_initial

        # R = A @ u_initial
        # ratio = 1
        # # 定义时间步循环
        # sn = 1
        # for i in range(1, Max_step):
        #     u2 = spsolve(At, R)
        #     R = A @ u2 # 更新等式右边 R
        #     if i % ratio == 0:
        #         u[:,sn] = u2
        #         sn +=1
        # u_squared = u**2
        # error = (np.sum(u_squared, axis=0))**0.5

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"lamda_n_int_to_result_dataset_{timestamp}.pkl"
with open(filename, "wb") as f:
    pickle.dump(dataset, f)

print(f"✅ 数据集已保存，包含 {len(dataset)} 个样本。文件名是：{filename}")

⏱️ 7 samples have been generated (reaction coefficient is: 46.71650317069742*r^2). Time: 0.01 s
⏱️ 7900 samples have been generated (reaction coefficient is: 57.64165115076816*r^2). Time: 8.71 s
⏱️ 15793 samples have been generated (reaction coefficient is: 55.681137240398854*r^2). Time: 17.39 s
⏱️ 23657 samples have been generated (reaction coefficient is: 62.83930884318026*r^2). Time: 26.06 s
⏱️ 31534 samples have been generated (reaction coefficient is: 61.63953361722898*r^2). Time: 34.77 s
⏱️ 39417 samples have been generated (reaction coefficient is: 51.693952500226196*r^2). Time: 43.52 s
⏱️ 47270 samples have been generated (reaction coefficient is: 57.03184499064133*r^2). Time: 52.27 s
⏱️ 55155 samples have been generated (reaction coefficient is: 52.85033768768947*r^2). Time: 61.14 s
⏱️ 63041 samples have been generated (reaction coefficient is: 58.01418294573871*r^2). Time: 70.00 s
⏱️ 70907 samples have been generated (reaction coefficient is: 47.53493084255083*r^2). Time: 78.

In [ ]:
import pickle
# 读取数据集
filename = "lamda_n_int_to_kernel_dataset_20250330_224055.pkl"

with open(filename, "rb") as f:
    dataset = pickle.load(f)

print(f"样本数量: {len(dataset)}")

# 查看数据集的第一个样本
lambda_set = dataset[0]
for key, value in lambda_set.items():
    print(f"{key}: type = {type(value)}, shape = {getattr(value, 'shape', 'N/A')}")

# 单独查看每个 key 的值
print("lamda 值:")
print(lambda_set["lamda"])   # 这是一个 numpy 数组

print("\nn 值:")
print(lambda_set["n"])       # 这是一个整数

print("\nKernel 值:")
print(lambda_set["Kernel"])       # 这是一个 numpy 矩阵（数组）

样本数量: 78743
lamda: type = <class 'numpy.ndarray'>, shape = (91,)
n: type = <class 'int'>, shape = N/A
Kernel: type = <class 'numpy.ndarray'>, shape = (91, 91)
lamda 值:
[1.4259792e-03 1.2833812e-02 3.5649478e-02 6.9872975e-02 1.1550431e-01
 1.7254348e-01 2.4099047e-01 3.2084531e-01 4.1210797e-01 5.1477849e-01
 6.2885684e-01 7.5434297e-01 8.9123696e-01 1.0395389e+00 1.1992484e+00
 1.3703660e+00 1.5528913e+00 1.7468245e+00 1.9521655e+00 2.1689143e+00
 2.3970709e+00 2.6366355e+00 2.8876078e+00 3.1499879e+00 3.4237759e+00
 3.7089717e+00 4.0055757e+00 4.3135872e+00 4.6330061e+00 4.9638333e+00
 5.3060684e+00 5.6597114e+00 6.0247622e+00 6.4012203e+00 6.7890868e+00
 7.1883612e+00 7.5990429e+00 8.0211325e+00 8.4546309e+00 8.8995361e+00
 9.3558493e+00 9.8235703e+00 1.0302699e+01 1.0793236e+01 1.1295181e+01
 1.1808534e+01 1.2333294e+01 1.2869462e+01 1.3417038e+01 1.3976022e+01
 1.4546413e+01 1.5128213e+01 1.5721420e+01 1.6326035e+01 1.6942059e+01
 1.7569489e+01 1.8208328e+01 1.8858574e+01 1.952022

In [24]:
print(dataset[0]["lamda"].shape)

(91,)
